In [ ]:
import argparse
import sys
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
test =  tf.Variable(tf.random_uniform([784, 100],-0.1,0.1),name="W1")

In [ ]:
train_list = tf.trainable_variables()

In [ ]:
train_list

In [ ]:
train_list.remove(test)

In [ ]:
train_list

In [ ]:
train_list[0]

In [ ]:
class model():
    def __init__(self,profile="harmonic"):
        self.x = tf.placeholder(tf.float32, [None, 784])
        self.W1 = tf.Variable(tf.random_uniform([784, 100],-0.1,0.1),name="W1")
        self.b1 = tf.Variable(tf.random_uniform([100],-0.1,0.1),name="b1")
        self.W2 = tf.Variable(tf.random_uniform([100, 100],-0.1,0.1),name="W2")
        self.b2 = tf.Variable(tf.random_uniform([100],-0.1,0.1),name="b2")
        self.W3 = tf.Variable(tf.random_uniform([100, 10],-0.1,0.1),name="W3")
        self.b3 = tf.Variable(tf.random_uniform([10],-0.1,0.1),name="b3")
        
        if profile == "linear":
            self.r1 = tf.constant_initializer(np.linspace(1,0,num=100,endpoint=False))
            self.r2 = tf.constant_initializer(np.linspace(1,0,num=100,endpoint=False))
            self.r3 = tf.constant_initializer(np.linspace(1,0,num=10,endpoint=False))
        if profile == "harmonic":
            self.r1 = tf.constant_initializer(1.0/(np.arange(100)+1))
            self.r2 = tf.constant_initializer(1.0/(np.arange(100)+1))
            self.r3 = tf.constant_initializer(1.0/(np.arange(10)+1))
        else:
            
    def train_normal(self):
        
    def train_gamma(self):
        
        z1 = 
        y = tf.matmul(x, W) + b

In [ ]:
def main(_):
    # Import data
    mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)
    
    # Create the model
    x = tf.placeholder(tf.float32, [None, 784])
    W1 = tf.Variable(tf.zeros([784, 10]))
    b1 = tf.Variable(tf.zeros([10]))
    W2 = tf.Variable(tf.)
    y = tf.matmul(x, W) + b

    # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 10])
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
    
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    # Train
    for _ in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    # Test trained model
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels}))
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='/tmp/tensorflow/mnist/input_data',
                  help='Directory for storing input data')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)